### Startup imports

In [10]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.slurm_manager import slurm_manager
from autoqchem.helper_classes import slurm_status
from autoqchem.draw_utils import draw

import logging
logging.basicConfig(level=logging.INFO)

### Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [4]:
sm=slurm_manager(user='zuranski', host='della.princeton.edu')

### Provide some smiles string(s)

and copy into them into ```my_smiles``` variable. It's a list of strings (a Python list can be of any size)

In [5]:
my_smiles=[
    "C(C1C(C(C(C(O1)O)O)O)O)O",
    "c1ccc(F)cc1",
]

For a list of smiles, you can also read them out from a csv file in this way, assumed file name ```smiles_strings.csv```, also assuming the strings are in the first column, and there are no headers

In [6]:
# my_smiles = pd.read_csv("smiles_strings.csv", header=None, usecols=[0])[0].tolist()
# my_smiles

### Initialize the molecules and generate conformations

We will use some ```for``` loops to loop over the smiles

In [7]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, num_conf=10))

Draw some molecules

In [12]:
draw(mols[0].mol)

interactive(children=(Dropdown(description='confId', options=(0, 1, 2, 3, 4, 5, 6, 7), value=0), Output()), _d…

<function autoqchem.draw_utils._graph_conf(m, confId=0, energies=[])>

### Create gaussian jobs for each conformation

In [70]:
for mol in mols[1:]:
    sm.create_jobs_for_molecule(mol, light_basis_set='6-31G**', solvent='Ethanol')

In [81]:
# view the workflow status of all jobs
sm.get_job_stat(split_by_can=True)

### Submit the jobs to the server 

if you are not connected, it will ask for password + Duo

In [79]:
sm.submit_jobs()

Check the status of the jobs on the server

In [75]:
sm.squeue(summary=False)

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)


### Retrieve jobs

If there are some finished jobs the log files will be checked for completion and downloaded locally.

In [76]:
sm.retrieve_jobs()

INFO:autoqchem.slurm_manager:There are 0 running/pending jobs, 7 finished jobs.
INFO:autoqchem.slurm_manager:Retrieving log files of finished jobs.
INFO:autoqchem.slurm_manager:7 jobs finished successfully (all Gaussian steps finished normally). 0 jobs failed.


### Resubmit incomplete jobs

In [48]:
sm.resubmit_incomplete_jobs()

INFO:autoqchem.slurm_manager:There are no incomplete jobs to resubmit.


### Upload molecules to the database

In [78]:
sm.upload_done_molecules_to_db(tags=["test_AMZ"])

INFO:autoqchem.slurm_manager:There are 1 finished molecules ['C(C1C(C(C(C(O1)O)O)O)O)O'].
INFO:autoqchem.slurm_manager:Molecule C(C1C(C(C(C(O1)O)O)O)O)O has 0 / 8 duplicate conformers.
INFO:autoqchem.slurm_manager:Removing 0 / 8 jobs and log files that contain duplicate conformers.
INFO:autoqchem.slurm_manager:Uploaded descriptors to DB for smiles: C(C1C(C(C(C(O1)O)O)O)O)O, number of conformers: 8, DB molecule id 61bbad58bb6b37fd936249b9.
